In [2]:
from shared_code.utility.schemas.spark_table_schema import image_table_schema
from shared_code.utility.spark.set_environ import *


os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
set_azure_env()

spark = get_session('add-tokens-from-caption')

In [3]:
spark_df = spark.createDataFrame(spark.read.parquet("D:\\data\\processed\\reddit_images_vit_captioned.parquet").collect(),
								 schema=image_table_schema)

print(f"Total Number Of Records From Pre-Processing {spark_df.count()}")

display(spark_df.limit(10).toPandas())

Total Number Of Records From Pre-Processing 6501


,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,11090tg,D:\data\images\EarthPorn\vc1wh65itqha1.jpg,"Depoe Bay, OR USA [OC] 1080x1920",11090tg,Bpercell,https://i.redd.it/vc1wh65itqha1.jpg,NaN,/r/EarthPorn/comments/11090tg/depoe_bay_or_usa...,31c6df5a2815d8e5e8c85fe51b08460e,EarthPorn,A group of people are sitting on the shore of ...,True,vc1wh65itqha1.jpg,NaN,D:\data\images\EarthPorn\thumbnail\vc1wh65itqh...,False
1,training,11092s0,D:\data\images\SFWRedheads\8lb4z8n3uqha1.jpg,Morning hair xx,11092s0,demidiamond4,https://i.redd.it/8lb4z8n3uqha1.jpg,NaN,/r/SFWRedheads/comments/11092s0/morning_hair_xx/,afe1ae1ae3e663b0ca62cb3b19a5c3ef,SFWRedheads,A girl in a pink dress is smiling.,True,8lb4z8n3uqha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\8lb4z8n3u...,False
2,training,110961d,D:\data\images\CityPorn\2czgh0e5vqha1.jpg,St Paul’s Cathedral. London.,110961d,OkCardiologist2765,https://i.redd.it/2czgh0e5vqha1.jpg,NaN,/r/CityPorn/comments/110961d/st_pauls_cathedra...,ef3c3f82ec490f8d3ccc06948258a3b4,CityPorn,A man in a red coat is standing in front of a ...,True,2czgh0e5vqha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\2czgh0e5vqha...,False
3,training,1109a10,D:\data\images\SFWRedheads\hy9j0og8wqha1.jpg,Do you like my natural red curls?!,1109a10,ketunicorn,https://i.redd.it/hy9j0og8wqha1.jpg,NaN,/r/SFWRedheads/comments/1109a10/do_you_like_my...,9008fe70d17b904fcfc7c6a2232ad569,SFWRedheads,A girl in a pink dress is standing in front of...,True,hy9j0og8wqha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\hy9j0og8w...,False
4,training,1109ycm,D:\data\images\CityPorn\7beed5n54rha1.jpg,"Taipei, the Seoul of Taiwan (which makes Kaohs...",1109ycm,Ducky118,https://i.redd.it/7beed5n54rha1.jpg,NaN,/r/CityPorn/comments/1109ycm/taipei_the_seoul_...,848b795097dca410c409797c679ab4fa,CityPorn,A bird flies over the water.,True,7beed5n54rha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\7beed5n54rha...,False
5,training,110b0wt,D:\data\images\SFWNextDoorGirls\6piZs7j.jpg,"A real no makeup moment, cause this is how Sun...",110b0wt,princessxo699,https://i.imgur.com/6piZs7j.jpg,Feelin Happy 😃,/r/SFWNextDoorGirls/comments/110b0wt/a_real_no...,5ccd774afb17fb0c44460121ed9ce61d,SFWNextDoorGirls,A girl in a pink hat and sunglasses is talking...,True,6piZs7j.jpg,NaN,D:\data\images\SFWNextDoorGirls\thumbnail\6piZ...,False
6,training,110b1an,D:\data\images\CityPorn\rgotpnatzpha1.jpg,"Duisburg, Detroit of Germany",110b1an,Expensive-Team7416,https://i.redd.it/rgotpnatzpha1.jpg,NaN,/r/CityPorn/comments/110b1an/duisburg_detroit_...,3eb930bdf3fac4e5b16617edad9df65a,CityPorn,A bird flies over the water.,True,rgotpnatzpha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\rgotpnatzpha...,False
7,training,110bipg,D:\data\images\SFWNextDoorGirls\xywgncrbnrha1.jpg,enjoy your Sunday!,110bipg,trickyalice,https://i.redd.it/xywgncrbnrha1.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/110bipg/enjoy_you...,9475adbe05427c47772346f8ab672952,SFWNextDoorGirls,A woman in a pink shirt and jeans is talking t...,True,xywgncrbnrha1.jpg,NaN,D:\data\images\SFWNextDoorGirls\thumbnail\xywg...,False
8,training,110c2gi,D:\data\images\SFWRedheads\99hm504mtrha1.jpg,Happy sunday! Sending greetings! ❤️,110c2gi,GingeMariia,https://i.redd.it/99hm504mtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2gi/happy_sunday_s...,20ced2827eb9e9cd3464835306cfd075,SFWRedheads,A girl in a pink shirt and blue jeans is stand...,True,99hm504mtrha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\99hm504mt...,False
9,training,110c2vo,D:\data\images\SFWRedheads\kkerxboqtrha1.jpg,The kind of girl you can have a beer with.,110c2vo,MissKatiesss,https://i.redd.it/kkerxboqtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2vo/the_kind_of_gi...,f53572296a0b8fb6654808866440316a,SFWRedheads,A woman in a red dress is talking to a man in ...,True,kkerxboqtrha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\kkerxboqt...,False


In [4]:
import nltk
from nltk import TweetTokenizer
from nltk import sent_tokenize
from collections import OrderedDict
from pyspark.sql.functions import udf

def get_nlk_tokens(text: str):
	try:
		tokenizer = TweetTokenizer()

		first_sentence = sent_tokenize(text)[0]

		# remove numbers and tokenize the text
		tokenized = tokenizer.tokenize(first_sentence.translate({ord(ch): None for ch in '0123456789'}))

		tokenized = [i for i in tokenized if len(i) > 1]

		tokenized = list(OrderedDict.fromkeys(tokenized))

		pos_tagged_text = nltk.pos_tag(tokenized)

		# Extract all nouns, verbs and adverbs and append to the existing
		prompt_keywords = [i[0] for i in pos_tagged_text if i[1][:2] in ['NN', 'VB', 'RB']]

		prompt_keywords = list(OrderedDict.fromkeys(prompt_keywords))

		return pos_tagged_text
	except Exception as e:
		print(e)
		return [[]]


def get_tokens_for_caption(caption: str):
	if caption == 'NaN':
		return []
	if caption == "":
		return []
	else:
		tokens = get_nlk_tokens(caption)
		return tokens

In [5]:
from shared_code.utility.schemas.spark_table_schema import tokenize_caption_schema, \
	image_table_schema_with_caption_tokens

get_caption_image_name_udf = udf(get_tokens_for_caption, tokenize_caption_schema)

updated_captions_collection = spark_df\
	.withColumn("caption_tokens", get_caption_image_name_udf("updated_caption"))\
	.withColumn("text_tokens", get_caption_image_name_udf("text"))\
	.collect()

# out_frame = spark.createDataFrame(updated_captions_collection, schema=image_table_schema)
out_frame = spark.createDataFrame(updated_captions_collection, schema=image_table_schema_with_caption_tokens)

print(f"Total Number Of Records From Pre-Processing {out_frame.count()}")

Total Number Of Records From Pre-Processing 6501


In [7]:
display(out_frame.where('size(caption_tokens) > 0').limit(10).toPandas())

,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated,caption_tokens,text_tokens
0,training,110jlij,D:\data\images\sfwpetite\awf8v23xrtha1.jpg,big smile,110jlij,shoot_score_dribble,https://i.redd.it/awf8v23xrtha1.jpg,None,/r/sfwpetite/comments/110jlij/big_smile/,6f4944cce46ba34dbdf8e47e3f99ad31,sfwpetite,A girl in a pink bathing suit is standing in f...,True,awf8v23xrtha1.jpg,a beautiful young woman posing for a picture i...,D:\data\images\sfwpetite\thumbnail\awf8v23xrth...,False,"[(beautiful, JJ), (young, JJ), (woman, NN), (p...","[(big, JJ), (smile, NN)]"
1,training,110jpcg,D:\data\images\sfwpetite\2pdd2lu6bsha1.jpg,Ripcurl,110jpcg,benderod,https://i.redd.it/2pdd2lu6bsha1.jpg,None,/r/sfwpetite/comments/110jpcg/ripcurl/,1fa72c4a1d3c76143db01e26d7a3f8dc,sfwpetite,A blonde haired blonde haired blonde haired bl...,True,2pdd2lu6bsha1.jpg,a woman in a bikini sitting in a car,D:\data\images\sfwpetite\thumbnail\2pdd2lu6bsh...,False,"[(woman, NN), (in, IN), (bikini, NN), (sitting...","[(Ripcurl, NN)]"
2,training,110ryuu,D:\data\images\Faces\avkbiiw6jvha1.jpg,Chapter 29,110ryuu,yoooooooobro,https://i.redd.it/avkbiiw6jvha1.jpg,Self,/r/Faces/comments/110ryuu/chapter_29/,7731577d8bcd7d10bf9850e3ebcd2e47,Faces,A man in a black shirt and jeans is talking to...,True,avkbiiw6jvha1.jpg,a man with a smile on his face posing for a pi...,D:\data\images\Faces\thumbnail\avkbiiw6jvha1.jpg,False,"[(man, NN), (with, IN), (smile, NN), (on, IN),...","[(Chapter, NN)]"
3,training,110s1my,D:\data\images\Faces\6s8l7qasjvha1.jpg,Smile everyone tomorrow is Monday so smile tod...,110s1my,BettyVespoli,https://i.redd.it/6s8l7qasjvha1.jpg,Self,/r/Faces/comments/110s1my/smile_everyone_tomor...,b2c2c140280591edfbe87197de199023,Faces,A girl in a pink dress is smiling and holding ...,True,6s8l7qasjvha1.jpg,a woman in a red dress is looking at the camera,D:\data\images\Faces\thumbnail\6s8l7qasjvha1.jpg,False,"[(woman, NN), (in, IN), (red, JJ), (dress, NN)...","[(Smile, NNP), (everyone, NN), (tomorrow, NN),..."
4,training,110s46x,D:\data\images\Faces\z3aohtzbkvha1.jpg,21F do you think Italians are cute ?,110s46x,Pelturr,https://i.redd.it/z3aohtzbkvha1.jpg,Self,/r/Faces/comments/110s46x/21f_do_you_think_ita...,c71cfe3717dc7ef69d7673a2d59fbd1d,Faces,A man and a woman are sitting on a bench.,True,z3aohtzbkvha1.jpg,a woman is sitting on the sidewalk while talki...,D:\data\images\Faces\thumbnail\z3aohtzbkvha1.jpg,False,"[(woman, NN), (is, VBZ), (sitting, VBG), (on, ...","[(do, VBP), (you, PRP), (think, VB), (Italians..."
5,training,110s84q,D:\data\images\Faces\02ybtok6lvha1.jpg,Can I be your new girlfriend? f22,110s84q,_ani_fan1,https://i.redd.it/02ybtok6lvha1.jpg,Self,/r/Faces/comments/110s84q/can_i_be_your_new_gi...,07c4493fcfa39af166893108a247e9cb,Faces,A girl in a pink dress is talking to another g...,True,02ybtok6lvha1.jpg,a woman in a pink shirt and a black tie,D:\data\images\Faces\thumbnail\02ybtok6lvha1.jpg,False,"[(woman, NN), (in, IN), (pink, NN), (shirt, NN...","[(Can, MD), (be, VB), (your, PRP$), (new, JJ),..."
6,training,110sfyw,D:\data\images\Faces\d99q53rt4uha1.jpg,will you make me smile?😊,110sfyw,blue-eyed_angel,https://i.redd.it/d99q53rt4uha1.jpg,Self,/r/Faces/comments/110sfyw/will_you_make_me_smile/,e4a16c262b67e4f576c5eae4c2f2fbf8,Faces,A girl in a pink dress is smiling.,True,d99q53rt4uha1.jpg,a woman smiles as she poses for a picture,D:\data\images\Faces\thumbnail\d99q53rt4uha1.jpg,False,"[(woman, NN), (smiles, VBZ), (as, IN), (she, P...","[(will, MD), (you, PRP), (make, VB), (me, PRP)..."
7,training,110shfq,D:\data\images\Faces\9ormpks5nvha1.jpg,F35 natural make up,110shfq,Main-Cheetah-4142,https://i.redd.it/9ormpks5nvha1.jpg,Self,/r/Faces/comments/110shfq/f35_natural_make_up/,ffc002fd3345af8781b3f390d35f7ffc,Faces,A girl in a red dress is standing in front of ...,True,9ormpks5nvha1.jpg,a woman with long hair wearing a pink dress,D:\data\images\Faces\thumbnail\9ormpks5

In [ ]:
out_frame.write.parquet("D:\\data\\processed\\reddit_images_processed_with_caption_tokens.parquet", mode='overwrite')